<a href="https://colab.research.google.com/github/Chayon58/Ensemble_Learning/blob/main/Ensember_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# ========================================
# STEP 0: Install dependencies
# ========================================
!pip install ultralytics scikit-learn albumentations opencv-python-headless matplotlib

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from ultralytics import YOLO
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ========================================
# STEP 1: Dataset Loader
# Supports jpg, jpeg, png
# ========================================
class LabeledImageDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.labels = pd.read_csv(csv_file)  # columns: filename,label
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_name = self.labels.iloc[idx, 0]
        img_path = os.path.join(self.img_dir, img_name)
        image = np.array(Image.open(img_path).convert("RGB"))
        label = int(self.labels.iloc[idx, 1])
        if self.transform:
            image = self.transform(image=image)["image"]
        return image, label

class UnlabeledImageDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.imgs = [f for f in os.listdir(img_dir) if f.lower().endswith(('.jpg','.jpeg','.png'))]
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.imgs[idx])
        image = np.array(Image.open(img_path).convert("RGB"))
        if self.transform:
            image = self.transform(image=image)["image"]
        return image, self.imgs[idx]

# ========================================
# STEP 2: User Config
# ========================================
img_dir = "/content/drive/MyDrive/images"   # folder with user-submitted images
csv_labels = None  # set CSV path if you have labels

aug_transform = A.Compose([
    A.Resize(224, 224),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=15, p=0.5),
    A.Normalize(),
    ToTensorV2()
])

if csv_labels:
    dataset = LabeledImageDataset(csv_labels, img_dir, transform=aug_transform)
    labeled = True
else:
    dataset = UnlabeledImageDataset(img_dir, transform=aug_transform)
    labeled = False

dataloader = DataLoader(dataset, batch_size=16, shuffle=True)
print("Number of images:", len(dataset))

# ========================================
# STEP 3: Models
# ========================================
class AutoEncoder(nn.Module):
    def __init__(self, latent_dim=128):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Flatten(),
            nn.Linear(224*224*3, latent_dim),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 224*224*3),
            nn.Sigmoid(),
            nn.Unflatten(1,(3,224,224))
        )
    def forward(self, x):
        z = self.encoder(x)
        out = self.decoder(z)
        return out, z

autoencoder = AutoEncoder().to(device)

if labeled:
    cnn = models.resnet18(pretrained=True)
    num_classes = len(set(pd.read_csv(csv_labels)['label']))
    cnn.fc = nn.Linear(cnn.fc.in_features, num_classes)
    cnn = cnn.to(device)

yolo = YOLO("yolov8n.pt")  # pretrained YOLOv8

# ========================================
# STEP 4: Train Autoencoder
# ========================================
print("Training Autoencoder...")
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=1e-3)

for epoch in range(2):
    autoencoder.train()
    for batch in dataloader:
        imgs = batch[0] if labeled else batch[0]
        imgs = imgs.to(device)
        outputs, _ = autoencoder(imgs)
        loss = criterion(outputs, imgs)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"[Autoencoder] Epoch {epoch+1}, Loss={loss.item():.4f}")

# ========================================
# STEP 5: Train CNN (if labeled)
# ========================================
if labeled:
    print("Training CNN...")
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(cnn.parameters(), lr=1e-4)
    for epoch in range(2):
        cnn.train()
        for imgs, labels in dataloader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = cnn(imgs)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(f"[CNN] Epoch {epoch+1}, Loss={loss.item():.4f}")

# ========================================
# STEP 6: YOLO Detection, Bounding Boxes, Masking
# ========================================
detection_confidences = []
masked_crops_all = []

for idx in range(len(dataset)):
    img_data, img_name = dataset[idx] if not labeled else dataset[idx]

    np_img = img_data.permute(1,2,0).cpu().numpy() if torch.is_tensor(img_data) else img_data
    np_img_vis = (np_img * 255).astype(np.uint8).copy()

    results = yolo.predict(np_img, verbose=False)

    crops = []
    confidences = []

    if results[0].boxes is not None and len(results[0].boxes) > 0:
        for i, box in enumerate(results[0].boxes.xyxy):
            x1, y1, x2, y2 = box.cpu().numpy().astype(int)
            conf = results[0].boxes.conf[i].cpu().numpy()
            crop = np_img[y1:y2, x1:x2]
            crop = cv2.resize(crop, (224,224))
            crops.append(crop)
            confidences.append(conf)

            # Draw bounding box
            cv2.rectangle(np_img_vis, (x1,y1), (x2,y2), (0,255,0), 2)
            cv2.putText(np_img_vis, f"{conf:.2f}", (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
    else:
        crop = cv2.resize(np_img, (224,224))
        crops.append(crop)
        confidences.append(0)

    masked_crops_all.append([torch.tensor(c).permute(2,0,1).float()/255.0 for c in crops])
    detection_confidences.append(np.mean(confidences))

    # Show image with bounding boxes
    plt.figure(figsize=(6,6))
    plt.imshow(np_img_vis)
    plt.title(f"{img_name} - {len(crops)} objects detected")
    plt.axis('off')
    plt.show()

# ========================================
# STEP 7: Detection Confidence Histogram
# ========================================
plt.figure(figsize=(8,5))
plt.hist(detection_confidences, bins=10, color='skyblue', edgecolor='black')
plt.xlabel("Detection Confidence")
plt.ylabel("Number of Images")
plt.title("Histogram of YOLO Detection Confidence per Image")
plt.show()

# ========================================
# STEP 8: Feature Extraction for Ensemble
# ========================================
def extract_auto(model, dataloader):
    model.eval()
    feats, labels = [], []
    with torch.no_grad():
        for batch in dataloader:
            if labeled:
                imgs, lbls = batch
            else:
                imgs, lbls = batch[0], None
            imgs = imgs.to(device)
            _, z = model(imgs)
            feats.append(z.cpu().numpy())
            if labeled: labels.append(lbls.numpy())
    return np.vstack(feats), (np.hstack(labels) if labeled else None)

def extract_cnn(model, dataloader):
    feats, labels = [], []
    model.eval()
    with torch.no_grad():
        for imgs, lbls in dataloader:
            imgs = imgs.to(device)
            out = model(imgs)
            probs = torch.softmax(out, dim=1).cpu().numpy()
            feats.append(probs)
            labels.append(lbls.numpy())
    return np.vstack(feats), np.hstack(labels)

print("Extracting features...")
auto_feats, y = extract_auto(autoencoder, dataloader)
cnn_feats, _ = (extract_cnn(cnn, dataloader) if labeled else (None,None))
yolo_feats = np.vstack([torch.stack(crops).view(len(crops),-1).numpy() for crops in masked_crops_all])

# ========================================
# STEP 9: Ensemble + Metrics
# ========================================
if labeled:
    print("Training meta-classifier...")
    X_parts = [auto_feats, yolo_feats]
    if cnn_feats is not None:
        X_parts.append(cnn_feats)
    X = np.hstack(X_parts)

    meta = LogisticRegression(max_iter=1000)
    meta.fit(X, y)
    preds = meta.predict(X)

    acc = accuracy_score(y, preds)
    prec, recall, f1, _ = precision_recall_fscore_support(y, preds, average="weighted")
    print(f"✅ Ensemble Results: Acc={acc:.4f}, Precision={prec:.4f}, Recall={recall:.4f}, F1={f1:.4f}")
else:
    print("✅ Unsupervised run complete (Autoencoder + YOLO features extracted).")
